# VITS

In [ ]:
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write

def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [ ]:
hps_mt = utils.get_hparams_from_file("./configs/genshin.json")

In [ ]:
npcList = ['派蒙', '凯亚', '安柏', '丽莎', '琴', '香菱', '枫原万叶',
           '迪卢克', '温迪', '可莉', '早柚', '托马', '芭芭拉', '优菈',
           '云堇', '钟离', '魈', '凝光', '雷电将军', '北斗',
           '甘雨', '七七', '刻晴', '神里绫华', '戴因斯雷布', '雷泽',
           '神里绫人', '罗莎莉亚', '阿贝多', '八重神子', '宵宫',
           '荒泷一斗', '九条裟罗', '夜兰', '珊瑚宫心海', '五郎',
           '散兵', '女士', '达达利亚', '莫娜', '班尼特', '申鹤',
           '行秋', '烟绯', '久岐忍', '辛焱', '砂糖', '胡桃', '重云',
           '菲谢尔', '诺艾尔', '迪奥娜', '鹿野院平藏']

net_g_mt = SynthesizerTrn(
    len(symbols),
    hps_mt.data.filter_length // 2 + 1,
    hps_mt.train.segment_size // hps_mt.data.hop_length,
    n_speakers=hps_mt.data.n_speakers,
    **hps_mt.model).cuda()
_ = net_g_mt.eval()


_ = utils.load_checkpoint("./G_809000.pth", net_g_mt, None)

In [ ]:
from tqdm import tqdm
import numpy

In [ ]:
sentences=[
    "朕深鉴于世界大势及帝国之现状，欲采取非常之措施，收拾时局，兹告尔等臣民，",
    "朕已饬令帝国政府通告美、英、中、苏四国，愿接受其联合公告。",
    "盖谋求帝国臣民之康宁，同享万邦共荣之乐，斯乃皇祖皇宗之遗范，亦为朕所眷眷不忘者；",
    "前者，帝国之所以向美英两国宣战，实亦为希求帝国之自存于东亚之安定而出此，至如排斥他国之主权，侵犯他国之领土，固非朕之本志；",
    "然交战已阅四载，虽陆海将兵勇敢善战，百官有司励精图治，一亿众庶克己奉公，各尽所能，而战局并未好转，世界大势亦不利于我。",
    "加之，敌方最近使用残酷之炸弹，频杀无辜，惨害所及，实难逆料；",
    "如仍继续作战，则不仅导致我民族之灭亡；并将破坏人类之文明。",
    "如此，则朕将何以保全亿兆赤子，陈谢于皇祖皇宗之神灵乎！此朕所以饬帝国政府接受联合公告者也。",
    "朕对于始终与帝国同为东亚解放而努力之诸盟邦，不得不深表遗憾；",
    "念及帝国臣民之死于战阵，殉于职守，毙于非命者及其遗属，则五脏为之俱裂；",
    "至于负战伤，蒙战祸，失家业者之生计，亦朕所深为轸念者也；",
    "今后帝国所受之苦固非寻常，朕亦深知尔等臣民之衷情，然时运之所趋，朕欲忍所难忍，耐所难耐，以为万世之太平。",
    "朕于兹得以维护国体，信倚尔等忠良臣民之赤诚，并常与尔等臣民同在。",
    "若夫为情所激，妄滋事端，或者同胞互相排挤，扰乱时局；因而迷误大道，失信义于世界，此朕所深戒。",
    "宜举国一致，子孙相传，确信神州之不灭。",
    "念任重而道远，倾全力于将来之建设，笃守道义，坚定志操，誓必发扬国体之精华，不致落后于世界之进化，望尔等臣民善体朕意。",
]

    
audio_mt = numpy.array([])
speaker="钟离"
for sentence in tqdm(sentences,ncols=80):
    stn_tst_mt = get_text(sentence.replace("\n", "").replace('\u3000',''), hps_mt)
    with torch.no_grad():
        x_tst_mt = stn_tst_mt.cuda().unsqueeze(0)
        x_tst_mt_lengths = torch.LongTensor([stn_tst_mt.size(0)]).cuda()
        sid_mt = torch.LongTensor([npcList.index(speaker)]).cuda()
        audio_mt =numpy.concatenate((audio_mt,net_g_mt.infer(x_tst_mt, x_tst_mt_lengths, sid=sid_mt, noise_scale=.4, noise_scale_w=.8, length_scale=1.2)[0][0,0].data.cpu().float().numpy()))
ipd.display(ipd.Audio(audio_mt, rate=hps_mt.data.sampling_rate, normalize=False))


In [ ]:
sentences2=[
    "朕深鉴世界大势与帝国现状、欲以非常措置收拾时局、兹告忠良尔臣民：",
    "朕使帝国政府、对米、英、中、苏四国、旨通告其受诺共同宣言。",
    "抑图帝国臣民康宁、偕万邦共荣之乐者、皇祖皇宗之遗范、而朕之所拳拳不措也。",
    "曩所以宣战米英二国，亦实出庶几乎帝国自存与东亚之安定。",
    "如排他国主权、侵领土者，固非朕志。然交战已阅四岁、朕陆海将兵之勇战、朕百僚有司之励精、朕一亿众庶之奉公、各不拘于尽最善、而战局必不好转、世界大势亦非利我。",
    "加之敌新使用残虐爆弹、频杀伤无辜、惨害所及、真至不可测。而尚继续交战、终非但招来我民族之灭亡、延可破却人类文明。",
    "如斯、朕何以保亿兆赤子、谢于皇祖皇宗之神灵哉。是至朕所以使帝国政府应共同宣言也。",
    "朕对帝国与共终始协力于东亚解放诸盟邦、不得不表遗憾之意。",
    "致想帝国臣民、死于战阵、殉于职域、毙于非命者、及其遗族、五内为裂。",
    "且至负战伤、蒙灾祸、失家业者之厚生、朕之所深轸念也。",
    "惟今后帝国之受苦难、固非寻常、尔臣民之衷情、朕善知之。",
    "然朕时运所趋、堪难堪、忍难忍、欲以为万世开太平。",
    "朕兹得护持国体、信倚忠良尔臣民之赤诚、常与尔臣民共在。",
    "若夫情之所激、滥滋事端、或如同胞排挤、互乱时局、为误大道、失信义于世界者、朕最戒之。",
    "宜举国一家、子孙相传、确信神州之不灭、念任重而道远、倾总力于将来之建设、笃道义、巩志操、誓发扬国体之精华、可期不后于世界之进运矣。",
    "尔臣民、其克体朕意！"
]

audio_mt = numpy.array([])
speaker="雷电将军"
for sentence in tqdm(sentences,ncols=80):
    stn_tst_mt = get_text(sentence.replace("\n", "").replace('\u3000',''), hps_mt)
    with torch.no_grad():
        x_tst_mt = stn_tst_mt.cuda().unsqueeze(0)
        x_tst_mt_lengths = torch.LongTensor([stn_tst_mt.size(0)]).cuda()
        sid_mt = torch.LongTensor([npcList.index(speaker)]).cuda()
        audio_mt =numpy.concatenate((audio_mt,net_g_mt.infer(x_tst_mt, x_tst_mt_lengths, sid=sid_mt, noise_scale=.4, noise_scale_w=.8, length_scale=1.2)[0][0,0].data.cpu().float().numpy()))
ipd.display(ipd.Audio(audio_mt, rate=hps_mt.data.sampling_rate, normalize=False))

In [ ]:
sentence="轮回，我俯视万古，见惯生死！谁在轮回？仙王斩掉都只能成粪土，谁敢在我面前轮回？轮回只是一个美丽的笑话。真强者，从不信轮回！ "
speakers=['派蒙','派蒙','派蒙','派蒙','派蒙']
speakers2=['派蒙', '丽莎', '琴', '香菱','可莉', '早柚', '芭芭拉', '优菈', '云堇','凝光', '雷电将军', '北斗',
          '甘雨', '七七', '刻晴', '神里绫华', '罗莎莉亚', '八重神子', '宵宫', '九条裟罗', '夜兰', '珊瑚宫心海', 
          '女士', '莫娜', '申鹤', '烟绯', '久岐忍', '辛焱', '砂糖', '胡桃', '菲谢尔', '诺艾尔', '迪奥娜']
audio_mt = []
stn_tst_mt = get_text(sentence.replace("\n", "").replace('\u3000',''), hps_mt)
for speaker in tqdm(speakers):
    with torch.no_grad():
        x_tst_mt = stn_tst_mt.cuda().unsqueeze(0)
        x_tst_mt_lengths = torch.LongTensor([stn_tst_mt.size(0)]).cuda()
        sid_mt = torch.LongTensor([npcList.index(speaker)]).cuda()
        audio_mt.append(net_g_mt.infer(x_tst_mt, x_tst_mt_lengths, sid=sid_mt, noise_scale=.4, noise_scale_w=.8, length_scale=1.2)[0][0,0].data.cpu().float().numpy())

max=0
for audio in audio_mt:
    print(len(audio),end = " ")
    if len(audio)>max:
        max=len(audio)
print("\n\nmax="+str(max))

for i in range(len(audio_mt)):
    pad=max-len(audio_mt[i])
    audio_mt[i]=numpy.pad(
        audio_mt[i],
        (0,pad),
        'linear_ramp'
    )

sum= numpy.zeros([max], dtype=int)
for audio in audio_mt:
    sum=sum+audio
ipd.display(ipd.Audio(sum, rate=hps_mt.data.sampling_rate, normalize=True))